In [1]:
!uv pip install fiftyone==1.11.0

Using Python 3.11.14 environment at: /home/vanessa/.local/micromamba/envs/handsoncv
Resolved 110 packages in 76ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/10.77 MiB           
⠙ Preparing packages... (0/1)------------------- 16.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 32.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 48.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 60.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 76.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 92.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 108.44 KiB/10.77 MiB        
⠙ Preparing packages... (0/1)------------------- 124.44 KiB/10.77 MiB        
⠙ Preparing packages... (0/1)------------------- 140.44 KiB/10.

In [1]:
import os
os.environ["FIFTYONE_DISABLE_NOTEBOOK_APP"] = "1"

import fiftyone as fo
fo.app_config.enable_notebook_app = True

In [ ]:
PATH = "~/Documents/repos/Applied-Hands-On-Computer-Vision/Assignment-2/data/" #"~/mnt/gdrive/path/to/your/assessment_folder" # Update this path to where the data is
MOUNTED_PATH = os.path.expanduser(PATH)

dataset_name = "cilp_assessment"
if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)
dataset = fo.Dataset(dataset_name)
dataset.add_group_field("group", default="rgb")

classes = ["cubes", "spheres"]
samples = []

for label in classes:
    class_dir = os.path.join(MOUNTED_PATH, label)
    rgb_dir = os.path.join(class_dir, "rgb")
    lidar_dir = os.path.join(class_dir, "lidar")
    
    filenames = sorted([f for f in os.listdir(rgb_dir) if f.endswith(('.png', '.jpg'))])
    
    for fname in filenames:
        group = fo.Group()
        
        rgb_sample = fo.Sample(filepath=os.path.join(rgb_dir, fname), group=group.element("rgb"))
        rgb_sample["ground_truth"] = fo.Classification(label=label)
        
        # Look for matching lidar file
        lidar_sample = fo.Sample(filepath=os.path.join(lidar_dir, fname), group=group.element("lidar"))
        lidar_sample["ground_truth"] = fo.Classification(label=label)
        
        samples.extend([rgb_sample, lidar_sample])

dataset.add_samples(samples)

session = fo.launch_app(dataset)

# Print the instructions for the user in case the app does not launch in the notebook cell
print(f"\nOpen the App.")
print(f"1. Click the 'Views' icon (bookmark symbol) to see 'test_low_conf' and 'test_suspicious'.")
print(f"2. URL: {session.url}")

 100% |█████████████| 34244/34244 [16.4s elapsed, 0s remaining, 2.2K samples/s]      



Open the App.
1. Click the 'Views' icon (bookmark symbol) to see 'test_low_conf' and 'test_suspicious'.
2. URL: http://localhost:5151/



Could not connect session, trying again in 10 seconds

